Data Cleaner
==============

**Author:** *Nicolas Haase*

## Data Import

In [17]:
import pandas as pd

# Load data into a DataFrame
df = pd.read_csv('/Users/nico/Developments/strategic-flight-prediction/data/flight_data.csv')

/var/folders/k_/7tyv0nps0tn9zv3mhhdms5y80000gn/T/ipykernel_13735/1834435320.py:4: DtypeWarning: Columns (48,69,76,77,84,85,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/nico/Developments/strategic-flight-prediction/data/flight_data.csv')


## Data Information

In [1]:
print(df.shape)
print("=======================")
df.info()

NameError: name 'df' is not defined

## Feature Selection

In [19]:
# Drop the last 46 columns
df = df.iloc[:, :-46]

# Create a boolean mask to filter rows where "cancelled" or "diverted" is equal to one
mask = (df['Cancelled'] != 1) & (df['Diverted'] != 1)

# Apply the mask to the DataFrame to filter the rows
df_filtered = df[mask]

# df_filtered now contains rows where "cancelled" and "diverted" are not equal to one

In [20]:
columns_to_remove = [
    "Quarter", "DOT_ID_Reporting_Airline", "IATA_CODE_Reporting_Airline",
    "OriginAirportSeqID", "OriginCityMarketID", "OriginCityName", "OriginState",
    "OriginStateFips", "OriginStateName", "OriginWac", "DestAirportSeqID",
    "DestCityMarketID", "DestCityName", "DestState", "DestStateFips",
    "DestStateName", "DestWac", "DepDelayMinutes", "ArrDelayMinutes",
    "FirstDepTime", "TotalAddGTime", "LongestAddGTime", "Cancelled", "Diverted", "CancellationCode", "Flights"
]

# Remove the specified columns and reset the index
df_filtered = df_filtered.drop(columns=columns_to_remove)
df_filtered.reset_index(drop=True, inplace=True)

In [22]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13119923 entries, 0 to 13119922
Data columns (total 39 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Year                             int64  
 1   Month                            int64  
 2   DayofMonth                       int64  
 3   DayOfWeek                        int64  
 4   FlightDate                       object 
 5   Reporting_Airline                object 
 6   Tail_Number                      object 
 7   Flight_Number_Reporting_Airline  int64  
 8   OriginAirportID                  int64  
 9   Origin                           object 
 10  DestAirportID                    int64  
 11  Dest                             object 
 12  CRSDepTime                       int64  
 13  DepTime                          float64
 14  DepDelay                         float64
 15  DepDel15                         float64
 16  DepartureDelayGroups             float64
 17  DepTim

## Atlanta Airport

In [23]:
# Filter for flights with 10397 as the origin or destination id
atl_origin_id = 10397
df_atl = df_filtered[(df_filtered['OriginAirportID'] == atl_origin_id) | (df_filtered['DestAirportID'] == atl_origin_id)]
df_atl.reset_index(drop=True, inplace=True)

df_atl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276177 entries, 0 to 1276176
Data columns (total 39 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   Year                             1276177 non-null  int64  
 1   Month                            1276177 non-null  int64  
 2   DayofMonth                       1276177 non-null  int64  
 3   DayOfWeek                        1276177 non-null  int64  
 4   FlightDate                       1276177 non-null  object 
 5   Reporting_Airline                1276177 non-null  object 
 6   Tail_Number                      1276177 non-null  object 
 7   Flight_Number_Reporting_Airline  1276177 non-null  int64  
 8   OriginAirportID                  1276177 non-null  int64  
 9   Origin                           1276177 non-null  object 
 10  DestAirportID                    1276177 non-null  int64  
 11  Dest                             1276177 non-null 

## Top 5 Airports

In [24]:
# List of top ten airport codes to filter for
airport_codes = ['ATL', 'DFW', 'DEN', 'ORD', 'LAX']

# Create a DataFrame containing 'Origin' values
origin_df = pd.DataFrame({'Origin': airport_codes})

# Merge the 'Origin' DataFrame with the original DataFrame to get 'OriginAirportID'
result_df = pd.merge(origin_df, df_filtered[['Origin', 'OriginAirportID']], on='Origin', how='left')

# Extract only the unique values of the 'OriginAirportID' column
airport_ids = result_df['OriginAirportID'].unique()

result_df.info()
print(airport_ids)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2615525 entries, 0 to 2615524
Data columns (total 2 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   Origin           object
 1   OriginAirportID  int64 
dtypes: int64(1), object(1)
memory usage: 39.9+ MB
[10397 11298 11292 13930 12892]


In [25]:
# Use the isin() method to filter the DataFrame based on the list of airport ids
df_top_ten = df_filtered[(df_filtered['OriginAirportID'].isin(airport_ids)) | (df_filtered['DestAirportID'].isin(airport_ids))]

df_top_ten.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4991757 entries, 718 to 13119922
Data columns (total 39 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Year                             int64  
 1   Month                            int64  
 2   DayofMonth                       int64  
 3   DayOfWeek                        int64  
 4   FlightDate                       object 
 5   Reporting_Airline                object 
 6   Tail_Number                      object 
 7   Flight_Number_Reporting_Airline  int64  
 8   OriginAirportID                  int64  
 9   Origin                           object 
 10  DestAirportID                    int64  
 11  Dest                             object 
 12  CRSDepTime                       int64  
 13  DepTime                          float64
 14  DepDelay                         float64
 15  DepDel15                         float64
 16  DepartureDelayGroups             float64
 17  DepTimeBlk

## Save Cleaned Data

In [26]:
# Save data to .csv files
df_filtered.to_csv('../data/cleaned/flight_data_cleaned.csv', index=False)  # Set index to False to avoid saving the index as a column
df_atl.to_csv('../data/cleaned/atl_cleaned.csv', index=False)
df_top_ten.to_csv('../data/cleaned/top_five_cleaned.csv', index=False)